# TCP Socket application with MicroK8s on Ubuntu 20.04

"MicroK8s is a powerful, lightweight, reliable production-ready Kubernetes distribution. It is an enterprise-grade Kubernetes distribution that has a small disk and memory footprint while offering carefully selected add-ons out-the-box, such as Istio, Knative, Grafana, Cilium and more. Whether you are running a production environment or interested in exploring K8s, MicroK8s serves your needs," see [Introduction to MicroK8s](https://ubuntu.com/blog/introduction-to-microk8s-part-1-2).

### The model

AccountReceivable and GeneralLedger are microservices. We use tcp socket protocol for communication.

To create statefulset pods we used Suyash Mohan article [Setting up PostgreSQL Database on Kubernetes](https://medium.com/@suyashmohan/setting-up-postgresql-database-on-kubernetes-24a2a192e962) as a guideline.
```
                         Store
                           ↕
                       Counter (cnt)
                           ↓
    testpluto.jl        Invoice Nbr
          ↓                ↓
      Orders/ → AccountsReceivable (ar) → Entries → GeneralLedger (gl)
  BankStatements           ↕                               ↕
                         Store                           Store
                  Unpaid/PaidInvoices            Journal/GeneralLedger
```

### Check whether ar-statefulset, gl-statefullset and cnt-statefullset pods are running

Run the next command from the terminal:

**microk8s.kubectl get pods -n socket-ns**

```
NAME                READY   STATUS    RESTARTS   AGE
cnt-statefulset-0   2/2     Running   0          160m
gl-statefulset-0    2/2     Running   0          159m
gl-statefulset-1    2/2     Running   0          159m
ar-statefulset-1    2/2     Running   0          98m
ar-statefulset-0    2/2     Running   0          97m
```

### If pods are not running use next commands from the terminal
- Install microk8s: **sudo snap install microk8s --classic**
- Enable microk8s build-in apps: **microk8s enable registry dashboard dns istio storage**
- Clone files from GitHub: **git clone https://github.com/rbontekoe/ar.git**
- Enter the folder with the downloaded files: **cd ar**
- Download Julia 1.6.5: **curl -O https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.5-linux-x86_64.tar.gz**
- Create accounts receivable image: **docker build --no-cache -f ar.Dockerfile -t localhost:32000/i_ar:v1.0.16 .**
- Copy to local registry: **docker push localhost:32000/i_ar:v1.0.16**
- Create general ledger image: **docker build --no-cache -f gl.Dockerfile -t localhost:32000/i_gl:v1.0.4 .**
- Copy to local registry: **docker push localhost:32000/i_gl:v1.0.4**
- Create counter image: **docker build --no-cache -f cnt.Dockerfile -t localhost:32000/i_cnt:v1.0.1 .**
- Copy to local registry: **docker push localhost:32000/i_cnt:v1.0.1**
- **microk8s.kubectl apply -f 1-ar-storage.yaml**
- **microk8s.kubectl apply -f 2-gl-storage.yaml**
- **microk8s.kubectl apply -f 3-cnt-storage.yaml**

## Connect to AppliAR.jl, create and process the orders

In [1]:
import Pkg; Pkg.add(url="https://github.com/rbontekoe/AppliAR")

### Load the files

In [2]:
using Sockets, Serialization, AppliSales, AppliAR, AppliGeneralLedger, DataFrames, Query

### Delete the old data files

Go to the terminal and delete the next files if you want a fresh start:

```
sudo rm -f generalledger.txt paid-invoices.txt unpaid-invoices.txt journal.txt seqnbr.txt
```

### Connect to AppliAR.jl, create and process the orders

In [3]:
clientside = connect(ip"127.0.0.1", 30012) # connect to accounts receivable pod

TCPSocket(RawFD(47) open, 0 bytes waiting)

In [19]:
sales = AppliSales.process() # create sales orders
serialize(clientside, sales) # send orders to account receivable

In [20]:
r0 = AppliGeneralLedger.read_from_file("/var/data-ar/generalledger.txt")
df1 = r0 |> @filter(_.accountid == 1300) |> DataFrame
df1[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]

,accountid,customerid,invoice_nbr,debit,credit,descr
,Int64,String,String,Float64,Float64,String
1,1300,Scrooge Investment Bank,1001,1210.0,0.0,Learn Smiling
2,1300,Duck City Chronicals,1002,2420.0,0.0,Learn Smiling
3,1300,Donalds Hardware Store,1003,1210.0,0.0,Learn Smiling


### Load and process the bank statements

In [21]:
stms = AppliAR.read_bank_statements("./bank-kubernetes.csv") # retrieve the bankstatements

2-element Vector{BankStatement}:
 BankStatement(Dates.Date("2020-01-15"), "Duck City Chronicals Invoice 1002", "NL93INGB", 2420.0)
 BankStatement(Dates.Date("2020-01-15"), "Donalds Hardware Store Bill 1003", "NL39INGB", 1210.0)

In [22]:
serialize(clientside, stms) # create paid invoices and update general ledger

### Display Accounts Receivable

Other accounts are:

```
8000 - Sales
1150 - Bank
4000 - VAT
1300 - Accounts Receivable
```

In [23]:
accountid = 1300;

In [24]:
r2 = AppliGeneralLedger.read_from_file("/var/data-ar/generalledger.txt")
df2 = r2 |> @filter(_.accountid == accountid) |> DataFrame
df2[:, [:accountid, :customerid, :invoice_nbr, :debit, :credit, :descr]]

,accountid,customerid,invoice_nbr,debit,credit,descr
,Int64,String,String,Float64,Float64,String
1,1300,Scrooge Investment Bank,1001,1210.0,0.0,Learn Smiling
2,1300,Duck City Chronicals,1002,2420.0,0.0,Learn Smiling
3,1300,Donalds Hardware Store,1003,1210.0,0.0,Learn Smiling
4,1300,Duck City Chronicals,1002,0.0,2420.0,Learn Smiling
5,1300,Donalds Hardware Store,1003,0.0,1210.0,Learn Smiling


### Display the status of the unpaid invoices

In [7]:
r1 = AppliAR.aging("/var/data-ar/unpaid-invoices.txt", "/var/data-ar/paid-invoices.txt")
result = DataFrame(r1)

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,1001,Scrooge Investment Bank,"Date(""2022-04-06"")",1210.0,Day(4)


In [4]:
@async while isopen(clientside)
    data = deserialize(clientside)
    println(data)
end

Task (runnable) @0x00007ff2192ddba0

In [5]:
serialize(clientside, "status")

Hello world!
